<a href="https://colab.research.google.com/github/Nightmare125/Compiler/blob/main/Week_long_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

122589 Maina Richard Gichomo

ICS 4A

Task 1

In Python:

In [ ]:
def is_comment(line):
    # Check if a line starts with common comment indicators
    comment_indicators = ['//', '#', '/*', '*']
    for indicator in comment_indicators:
        if line.strip().startswith(indicator):
            return True
    return False

def identify_lines(lines):
    results = []
    for line in lines:
        if is_comment(line):
            results.append("Comment: " + line.strip())
        else:
            results.append("Code: " + line.strip())
    return results

# Example usage:
if __name__ == "__main__":
    code_lines = [
        "int x = 10; // Initialize x to 10",
        "print('Hello, world!')  # This is a comment",
        "/* This is a multi-line",
        "   comment */",
        "for i in range(5):",
        "    print(i) # Print the value of i",
    ]

    results = identify_lines(code_lines)
    for result in results:
        print(result)


i.) Logic Used:

*   The is_comment function determines whether a line begins with a common comment indicator like "//," "#," "/," or "." A line is regarded as a comment if it begins with one of these markers. If not, it qualifies as code.

*   The is_comment function is used by the identify_lines function to detect whether each line in a list of lines is a comment or a piece of code.    Each result in the list of results, which it returns, has a string indicating whether the line in question is code or a comment.

ii.) Lexical and Syntax Analysis:


*   This logic uses lexical analysis when we look for common comment indications at the beginning of each line. We search for specific character combinations that are frequently used in programming languages to represent comments.

*   This fundamental comment identification mechanism does not explicitly involve syntax analysis because we are not parsing or analyzing the code's structure. To make a straightforward determination, we are merely searching for evidence of frequent comments at the beginning of lines.


Task 2

Python Program to Validate Identifiers:

In [ ]:
import re

def is_valid_identifier(identifier):
    # Use a regular expression to validate the identifier
    # An identifier should start with a letter or underscore, followed by letters, digits, or underscores.
    # It should not be a keyword or a reserved word.
    keywords = ["if", "else", "while", "for", "return", "int", "float", "char", "bool"]
    if identifier in keywords:
        return False
    return re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*$', identifier) is not None

# Example usage:
if __name__ == "__main__":
    identifier = input("Enter an identifier: ")
    if is_valid_identifier(identifier):
        print(f"'{identifier}' is a valid identifier.")
    else:
        print(f"'{identifier}' is not a valid identifier.")


To determine whether the provided identifier complies with the norms of most programming languages, this Python application employs a regular expression. It examines whether the identifier has letters, numbers, or underscores at the beginning, followed by a letter or underscore. The identification is also verified to ensure that it is neither a keyword or a reserved term.

Flex Specification File to Validate Identifiers:

In [ ]:
%{
#include <stdio.h>
%}

%%
[a-zA-Z_][a-zA-Z0-9_]* {
    printf("%s is a valid identifier.\n", yytext);
}
. {
    printf("%s is not a valid identifier.\n", yytext);
}
%%

int main() {
    yylex();
    return 0;
}
